In [2]:
import json
import pandas as pd
import numpy as np

'''bbox[0] = 'ymin'
   bbox[1] = 'xmin'
   bbox[2] = 'ymax'
   bbox[3] = 'xmax'
# 0: ymin, 1: xmin, 2: ymax, 3: xmax'''
'''
def class_text_to_int(row_label):
if row_label == 'hand':
return 1
elif row_label == 'grab_bottle_hand':
return 2
elif row_label == 'face':
return 3
elif row_label == 'mouth':
return 4
elif row_label == 'open_mouth':
return 5
elif row_label == 'bottle':
return 6
elif row_label == 'open_bottle':
return 7
elif row_label == 'cap':
return 8
elif row_label == 'pill':
return 9
elif row_label == 'cup':
return 10
else:
None
'''
colum_list = ['filename', 'width', 'height', 'obj_name', 'xmin', 'ymin', 'xmax', 'ymax', 'score', 'obj_idx']

# row_list = []

# per_file_per_object_info_list = []

df = pd.DataFrame(columns=colum_list)


with open('object_list.json') as json_file:
    json_data = json.load(json_file)
    file_name_list = list(json_data)         
    for filename in file_name_list:
        '''파일 1개당 json 정보'''
        '''case_1: json 정보 있는 경우 _ case_2: json 정보 없는 경우'''
        file_json_info = json_data[filename]
        if not file_json_info:
                obj_name = 'Null'
                obj_idx = 'Null'
                score = 'Null'
                ymin = 'Null'
                xmin = 'Null'
                ymax = 'Null'
                xmax = 'Null'
                width = 'Null'
                height = 'Null'
                df = df.append(pd.DataFrame([[filename+'_', width, height, obj_name, xmin, ymin, xmax, ymax, score, obj_idx]], columns=colum_list), ignore_index=True)
                print(df)
        else:
            '''파일 1개당 감지된_되어야하는_object 갯수'''
            obj_cnt_list = list(file_json_info)
            print(obj_cnt_list)
            '''per_obj_cnt 수정 필요(삭제)'''
            per_obj_cnt = len(obj_cnt_list)
            for i in obj_cnt_list:
                final_info = file_json_info[i]
                print(final_info)
                try:
                    obj_name = final_info['name']
                    obj_idx = final_info['obj']
                    score = final_info['score']
                    ymin = final_info['bbox'][0]
                    xmin = final_info['bbox'][1]
                    ymax = final_info['bbox'][2]
                    xmax = final_info['bbox'][3]
                    width = final_info['size'][0]
                    height = final_info['size'][1]
                    df = df.append(pd.DataFrame([[filename+'_', width, height, obj_name, xmin, ymin, xmax, ymax, score, obj_idx]], columns=colum_list), ignore_index=True)
                    print(df)
                except:
                    pass

        
        
        '''(위에서 json 정보 없는 경우에서 걸러짐) case_1: object 있는경우 // case_2: object 없는 경우'''
        
        '''json 내 obj, name, score, bbox, size'''



['0', '1', '2', '3', '4']
{'obj': 6, 'name': 'bottle', 'score': 0.9989, 'bbox': [62.279999999999994, 900.864, 263.66400000000004, 1190.784], 'size': [1280, 720]}
       filename width height obj_name     xmin   ymin      xmax     ymax  \
0  test_01.jpg_  1280    720   bottle  900.864  62.28  1190.784  263.664   

    score obj_idx  
0  0.9989       6  
{'obj': 1, 'name': 'hand', 'score': 0.9819, 'bbox': [196.84799999999998, 537.472, 719.496, 1267.456], 'size': [1280, 720]}
       filename width height obj_name     xmin     ymin      xmax     ymax  \
0  test_01.jpg_  1280    720   bottle  900.864   62.280  1190.784  263.664   
1  test_01.jpg_  1280    720     hand  537.472  196.848  1267.456  719.496   

    score obj_idx  
0  0.9989       6  
1  0.9819       1  
{'obj': 2, 'name': 'grab_bottle_hand', 'score': 0.8719, 'bbox': [176.84799999999998, 527.472, 739.496, 1367.456], 'size': [1280, 720]}
       filename width height          obj_name     xmin     ymin      xmax  \
0  test_01.jpg

In [29]:
df['per_obj_cnt'] = np.where(df['obj_idx'] != 'Null' , '1', '0')
df['model'] = 'answer_120'+'_'
df['result_per_obj_id_int'] = df.groupby(['filename','per_obj_cnt']).cumcount()+1
df['result_per_obj_id_str'] = df['result_per_obj_id_int']
'''
apply 쓰기 위해 int-> str 형 변환
'''

df['result_per_obj_id_str'] = df['result_per_obj_id_str'].apply(str)

'''
str 형태만 결합할 수 있음, int형태는 결합 안 됨
https://www.geeksforgeeks.org/create-a-new-column-in-pandas-dataframe-based-on-the-existing-columns/
'''

df['per_obj_pk'] = df.apply(lambda row: row.model + row.filename+ row.result_per_obj_id_str , axis=1)
df = df.reindex(columns=['per_obj_pk', 'model', 'filename', 'result_per_obj_id_str', 'obj_name', 'obj_idx', 'score', 'xmin', 'ymin', 'xmax', 'ymax', 'width', 'height', 'per_obj_cnt', 'result_per_obj_id_int'])

'''
cols = list(df.columns.values)
'''


'\ncols = list(df.columns.values)\n'

In [30]:
df

,per_obj_pk,model,filename,result_per_obj_id_str,obj_name,obj_idx,score,xmin,ymin,xmax,ymax,width,height,per_obj_cnt,result_per_obj_id_int
0,answer_120_0000_not_medi_079.jpg_1,answer_120_,0000_not_medi_079.jpg_,1,Null,Null,Null,Null,Null,Null,Null,Null,Null,0,1
1,answer_120_0001_not_medi_079.jpg_1,answer_120_,0001_not_medi_079.jpg_,1,hand,1,0.9212,462.72,86.76,1247.49,711.576,1280,720,1,1
2,answer_120_0002_not_medi_079.jpg_1,answer_120_,0002_not_medi_079.jpg_,1,hand,1,0.9042,430.08,69.696,1256.19,700.056,1280,720,1,1
3,answer_120_0003_not_medi_079.jpg_1,answer_120_,0003_not_medi_079.jpg_,1,hand,1,0.9362,451.456,83.808,1256.45,709.92,1280,720,1,1
4,answer_120_0004_not_medi_079.jpg_1,answer_120_,0004_not_medi_079.jpg_,1,hand,1,0.9305,598.4,50.2344,1273.34,720,1280,720,1,1
5,answer_120_0005_not_medi_079.jpg_1,answer_120_,0005_not_medi_079.jpg_,1,hand,1,0.9277,594.688,51.2856,1280,720,1280,720,1,1
6,answer_120_0006_not_medi_079.jpg_1,answer_120_,0006_not_medi_079.jpg_,1,hand,1,0.9471,475.52,66.3336,1257.86,718.128,1280,720,1,1
7,answer_120_0007_not_medi_079.jpg_1,answer_120_,0007_not_medi_079.jpg_,1,hand,1,0.926,456.064,70.9848,1264.13,712.944,1280,720,1,1
8,answer_120_0008_not_medi_079.jpg_1,answer_120_,0008_not_medi_079.jpg_,1,hand,1,0.9216,619.776,41.6952,1272.58,720,1280,720,1,1
9,answer_120_0009_not_medi_079.jpg_1,answer_120_,0009_not_medi_079.jpg_,1,Null,Null,Null,Null,Null,Null,Null,Null,Null,0,1


0    filename    0000_not_medi_079.jpg_
     width                         Null
     height                        Null
     obj_name                      Null
     xmin                          Null
     ymin                          Null
     xmax                          Null
     ymax                          Null
     score                         Null
     obj_idx                       Null
1    filename    0001_not_medi_079.jpg_
     width                         1280
     height                         720
     obj_name                      hand
     xmin                        462.72
     ymin                         86.76
     xmax                       1247.49
     ymax                       711.576
     score                       0.9212
     obj_idx                          1
2    filename    0002_not_medi_079.jpg_
     width                         1280
     height                         720
     obj_name                      hand
     xmin                        430.08
